# Handling Subprocess Errors

- External commands can fail in multiple ways: non-zero exit codes, missing executables, or hanging processes.  
- Using `subprocess.run(..., check=True)` shifts return-code checks into exceptions you can catch.  
- Specific exception types (`CalledProcessError`, `FileNotFoundError`, `TimeoutExpired`) let you distinguish failure modes and respond appropriately.

## subprocess.CalledProcessError Attributes

- `e.returncode`: the non-zero exit status of the command.  
- `e.cmd`: the exact command invoked (list or string form).  
- `e.stdout` / `e.output`: captured standard output, if `capture_output=True`.  
- `e.stderr`: captured standard error, if `capture_output=True`.  
- These attributes let you log or display detailed diagnostics when a command fails.  

In [29]:
import subprocess 

cmd = ["dir", "/nonexistent_path"]

try:
    subprocess.run(cmd, check=True, shell=True, capture_output=True, text=True)
except subprocess.CalledProcessError as e:
    print(f"Error executing command: {e.cmd}")  
    print(f"Return code: {e.returncode}")
    print(f"Standard output: {e.stdout}")
    print(f"Standard error: {e.stderr}")

Error executing command: ['dir', '/nonexistent_path']
Return code: 1
Standard output: 
Standard error: Parameter format not correct - "nonexistent_path".



## Handling FileNotFoundError

- If the executable itself isn’t in `PATH`, `subprocess.run()` raises `FileNotFoundError` before running.  
- Catching it separately lets you inform the user that a required tool isn’t installed, rather than treating it as a generic failure.  

In [37]:
import subprocess 

command = ["fakecmd", "--version"]

try:
    subprocess.run(command, check=True, capture_output=True, text=True)
except FileNotFoundError as e:
    print(f"Error executing command!")  
    print(f"  The command '{command[0]}' was not found on this system.")    

Error executing command!
  The command 'fakecmd' was not found on this system.


## Handling subprocess.TimeoutExpired

- Adding `timeout=<seconds>` to `subprocess.run()` kills the process if it runs too long.  
- A `TimeoutExpired` exception is raised, containing `cmd`, `timeout`, and any partial `stdout`/`stderr`.  
- Use this to prevent hung scripts and to implement retry or fallback logic.  

In [ ]:
import subprocess 

command = ["sleep", "10"]

try:
    subprocess.run(command,  timeout=2, shell=True, capture_output=True, text=True)
    print("Command executed successfully within timeout.")
except subprocess.TimeoutExpired as e:
    print("TimeoutExperied caught.")
    print(f"Command: {e.cmd}")
    print(f"Command timed out after {e.timeout} seconds.")

Command executed successfully within timeout.


## Recommended Error Handling Strategy

- Wrap `subprocess.run()` in a `try` block.  
- First catch `FileNotFoundError` to detect missing executables.  
- Next catch `subprocess.TimeoutExpired` if you use timeouts.  
- Then catch `subprocess.CalledProcessError` for non-zero exits.  
- Finally, if necessary, an `except Exception` block can log any other unexpected issues.  
- This layered approach keeps your script robust and your errors informative.  